# Building an ETL Pipeline

As the second part of the predict for Gather, you will need to build a pipeline of functions in python which does the following:

1. Function to connect to twitter and scrapes "Eskom_SA" tweets.
<br>
<br>
2. Cleans/Processes the tweets from the scraped tweets which will create a dataframe with two new columns using the following functions: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Hashtag Remover from Analyse Functions
<br>
<br>
3. Functions which connects to your SQL database and uploads the tweets into the table you store the tweets in the database.

In [0]:
# General:
import tweepy           # To consume Twitter's API
from tweepy import OAuthHandler
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys
import pandas as pd     # To handle data
import numpy as np      # For numerical computation
import json
# For plotting and visualization:
from IPython.display import display
import pyodbc


# Consumer and Access details

Fill in your Consumer and Access details you should have recieved when applying for a Twitter API. 

In [0]:
# Consumer:
CONSUMER_KEY    = 'uXfKji3NyWLDEGI3gXxx3tBNc'
CONSUMER_SECRET = 'eiuC2xhRfbmbY2gSplZOcL4dPSSUiqmB8SbLk6eGuZ2DaBaYQj'

# Access:
ACCESS_TOKEN  = '1219607153163603968-oOHm1E2uXjyBiBOkkKNExfAwvU4dVd'
ACCESS_SECRET = '3dZ7jyldTV1m1RxoVPv8VEOT92m2OwEPpF1Syj0j2Mu95'

In [0]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

# Function 1:

Write a function which:
- Scrapes _"Eskom_SA"_ tweets from Twitter. 

Function Specifications:
- The function should return a dataframe with the scraped tweets with just the "_Tweets_" and "_Date_". 
- Will take in the ```consumer key,  consumer secret code, access token``` and ```access secret code```.

NOTE:
The dataframe should have the same column names as those in your SQL Database table where you store the tweets.

In [41]:
def twitter_df():
  api = twitter_setup()
  all_tweets = []
  tweets = api.user_timeline(screen_name='ESKOM_SA', 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
  all_tweets.extend(tweets)
  
  outtweets = [[tweet.full_text.encode("utf-8").decode("utf-8"),tweet.created_at] 
              for idx,tweet in enumerate(all_tweets)]
  df = pd.DataFrame(outtweets,columns=['Tweets','Date'])
  df.set_index('Tweets')
  return df
twitter_df()

,Tweets,Date
0,"The Distribution Group Executive (GE), Monde ...",2020-03-03 11:47:46
1,It is a criminal offence to break or steal cov...,2020-03-03 10:30:00
2,Eskom's Visitors Centres’ facilities include i...,2020-03-03 08:30:00
3,#Eskom_EasternCape #MediaStatement\n\nEskom SU...,2020-03-03 07:45:10
4,@Esihleeh What is the issue that was reported?,2020-03-03 06:44:27
...,...,...
174,@whelanism If your power is off outside your l...,2020-02-21 09:07:44
175,@AnnajGoble @EskomSePush Is the area supplied ...,2020-02-21 09:04:22
176,@beatkonduktor Please report to Customer Servi...,2020-02-21 09:03:48
177,Eskom still on site https://t.co/ADebEssZO0,2020-02-21 08:08:56


# Function 2: Removing hashtags and the municipalities

Write a function which:
- Uses the function you wrote in the Analyse section to extract the hashtags and municipalities into it's own column in a new data frame. 

Function Specifications:
- The function should take in the pandas dataframe you created in Function 1 and return a new pandas dataframe. 

In [0]:
twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET )

In [42]:
def extract_municipality_hashtags(df):
    new_df = pd.DataFrame([])
    municipality_dict = { '@CityofCTAlerts' : 'Cape Town',
            '@CityPowerJhb' : 'Johannesburg',
            '@eThekwiniM' : 'eThekwini' ,
            '@EMMInfo' : 'Ekurhuleni',
            '@centlecutility' : 'Mangaung',
            '@NMBmunicipality' : 'Nelson Mandela Bay',
            '@CityTshwane' : 'Tshwane'}
  
    length = len(df['Tweets'])
    mun_list = []
    flag = 0
    for row in df['Tweets']:
        flag = 0
        for key in municipality_dict.keys():
            if key in row:
               mun_list.append(municipality_dict[key])
               flag = 1
            break
        if not flag:
          mun_list.append(np.nan)

    final_hash = []
    for row in df['Tweets']:
     final_hash.append([string for string in row.lower().split() if string[0][0] == '#'])
    final_hash = [np.nan if x == [] else x for x in final_hash]
    
    new_df['Tweets'] = df['Tweets']
    new_df['Date'] = df['Date']
    new_df['municipality'] = mun_list
    new_df['hashtags'] = final_hash
    
    return new_df
extract_municipality_hashtags(twitter_df())

,Tweets,Date,municipality,hashtags
0,"The Distribution Group Executive (GE), Monde ...",2020-03-03 11:47:46,NaN,NaN
1,It is a criminal offence to break or steal cov...,2020-03-03 10:30:00,NaN,NaN
2,Eskom's Visitors Centres’ facilities include i...,2020-03-03 08:30:00,NaN,NaN
3,#Eskom_EasternCape #MediaStatement\n\nEskom SU...,2020-03-03 07:45:10,NaN,"[#eskom_easterncape, #mediastatement]"
4,@Esihleeh What is the issue that was reported?,2020-03-03 06:44:27,NaN,NaN
...,...,...,...,...
174,@whelanism If your power is off outside your l...,2020-02-21 09:07:44,NaN,NaN
175,@AnnajGoble @EskomSePush Is the area supplied ...,2020-02-21 09:04:22,NaN,NaN
176,@beatkonduktor Please report to Customer Servi...,2020-02-21 09:03:48,NaN,NaN
177,Eskom still on site https://t.co/ADebEssZO0,2020-02-21 08:08:56,NaN,NaN


# Function 3: Updating SQL Database with pyODBC

Write a function which:
- Connects and updates your SQL database. 

Function Specifications:
- The function should take in a pandas dataframe created in Function 2. 
- Connect to your SQL database.
- Update the table you store your tweets in.
- Not return any output.

In [0]:
def pyodbc_twitter(connection, df, twitter_table):
  connection = pydobc.connect('Driver={SQL Server};'
                              'Server=IG-UMH-DSA12\SQLEXPRESS;'
                              'Database=gather_eskom;'
                              'Trusted_Connection=yes')
  cursor = connection.cursor()
  y = cursor.execute('SELECT * FROM gather_eskom.dbo.Station_Type')